In [11]:
import pandas as pd
import numpy as np
from scipy import sparse
from numba.decorators import jit,autojit
from scipy.spatial.distance import jaccard
from scipy.spatial.distance import cosine
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist

In [12]:
df=pd.read_csv('sample.csv',encoding='shift-jis',index_col=0)
df.head()

,USER_ID,ITEM_ID,CATEGORY
1,A,a,ズボン
2,A,c,シャツ
3,A,e,スカート
4,A,b,スニーカー
5,A,d,アウター


In [13]:
df.groupby('CATEGORY')['CATEGORY'].count()

CATEGORY
アウター     4
シャツ      2
スカート     2
スニーカー    3
ズボン      5
Name: CATEGORY, dtype: int64

▼jaccardで計算パターン

In [8]:
import time
start_time = time.clock()
result={}
#カテゴリをユニークにする
item=df['CATEGORY'].unique()

#カテゴリごとに抽出し、処理
for category in item:
    df1=df[df['CATEGORY']==category]#カテゴリで抽出
    df1=df1.pivot_table('CATEGORY',index='ITEM_ID',columns='USER_ID',aggfunc='count')#ピボットで集約
    df1=df1.fillna(0)#NAを０で埋めて０１のデータにする
    x=np.array(df1)#多次元のarrayに変換

    print(x)
    
    # 行ごと（=つまりアイテムごと）の距離を計算する
    d = pdist(x, 'jaccard')
    # 「類似度 = 1 - 距離」
    d = 1 - d

    d = squareform(d)


    # 自分自身は対象外にしたいので、優先度を下げる
    me = np.eye(d.shape[0])
    d = d - me

    # 類似度の高い順に5つ推薦対象とする
    for idx in range(x.shape[0]):
        item1_sim = d[:,idx]
        # print(item1_sim)
        item1_rel = []
        for i in range(item1_sim.shape[0]):
            item1_rel.append((df1.index[i], item1_sim[i]))
        item1_rel = sorted(item1_rel, key=lambda d:d[1], reverse=True)
        # 5件に絞る
        item1_rel = item1_rel[:5]
        result[df1.index[idx]] = item1_rel

    resultlist = category + "result.csv"
    results = pd.DataFrame(result)
    results.to_csv(resultlist)
    result={}

end_time = time.clock()
print ("time = %f" %(end_time-start_time))

[[ 1.  1.  2.  1.]
 [ 1.  1.  0.  0.]]
[[ 0.  1.  0.]
 [ 1.  0.  1.]]
[[1 1]]
[[1 1 1 1]]
time = 0.085416
